In [1]:
import os
import argparse
import json

import numpy as np
import easydict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from network.algebraic import AlgebraicTriangulation
from network.volumetric import VolumetricTriangulation
from utils import MSEsoft, MAELoss, MPJPE

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# arguments
args = easydict.EasyDict({"model_type":"alg",
                          "num_joints":17,
                          "num_cameras":4,
                          "agg_type":'sum'})

# tensor shape
B = 2 # batch size
C = 4 # number of cameras
J = 17 # number of joints
H, W = 256, 256 # height, weight of images

# set random tensor
batch_images = torch.rand([B, C, 3, H, W]).to(device)
batch_keypoints_3d_gt = torch.rand([B, J, 3]).to(device)
batch_proj_matrices = torch.rand([B, C, 3, 4]).to(device)
batch_pelvis = np.random.rand(B, J, 3)

Algebraic Triangulation Network

In [5]:
model = AlgebraicTriangulation(args)
model.to(device)

AlgebraicTriangulation(
  (backbone): Backbone(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
  

In [6]:
output = model(batch_images, batch_proj_matrices)
print(output.shape)

torch.Size([2, 17, 3])


Volumetric Triangulation Network

In [4]:
model = VolumetricTriangulation(args)
model.to(device)

VolumetricTriangulation(
  (backbone): Backbone(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
 

In [5]:
output = model(batch_images, batch_proj_matrices, batch_pelvis)
print(output.shape)

torch.Size([2, 17, 3])
